***
some experiments with roi geometries: selection and application of reference roi
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geemask



***
__selection of reference roi__

assume reference image\
assume roi to be specified in (an __integer__) number-of-pixels (size) in context of this image or projection

to obtain a symetrical roi, in this image (raster), around a (reference) point, :
- in case of an odd size, this reference point should be the center of a pixel
- in case of an even size, this reference point should be on a pixels border (intersection on grid)


__apply reference roi on target__

target image can be warped to align with this roi by reprojecting the target image
to the translated and rescaled projection of the reference image.

__heuristics target resolution__

target can specify its scale by stating its own (target) number-of-pixels in the reference roi.\
this number-of-pixels should be chosen
- large enough to avoid loss of data
- small enough to avoid redundant data
- (at least 4: however large the pixels are, they could always intersect in the roi)

e.g.: reference: S2 10m product, roi 32 pixels wide roi (hence approximatly 320m)\
consider target to be some PV 333m product.\
=> in the 320 x 320m roi, one could expect minimum 1, maximum 4 different PV 333m values\
=> roi ~32 ref pixels wide ~2 target pixels wide.

? heuristics: 
- choose target pixels ~  int( (ref pixels x ref res) / (target res) ) + 1 roughly (if more or less aligned)
- choose target pixels ~  int( (ref pixels x ref res) / (target res) ) + 2 safer (accounting for edges)

***

In [ ]:

def squarepixelboundsroi(eepoint, pixelsdiameter, eerefimage, verbose=False):
    size = round(pixelsdiameter)    #  "an integer" I said.
    size = max(size, 1)             #  preferably larger then 1 'reference image' pixel
    if (size %2) == 0:
        # even diameter
        eepoint = geeutils.pixelinterspoint(eepoint, eerefimage)
    else:
        # odd diameter
        eepoint = geeutils.pixelcenterpoint(eepoint, eerefimage)
    
    pixelsradius = size/2           # odd sizes: 1, 2, 3, ... - even sizes: 0.5, 1.5, 2.5, ...
    eeroi        = geeutils.squarerasterboundsroi(eepoint, pixelsradius, eerefimage)

    if verbose:
        print(f"squarepixelboundsroi pixelsdiameter({pixelsdiameter}) - pixelsradius({pixelsradius})")
        print(geeutils.szgeometryinfo(eepoint))
        print(geeutils.szgeometryinfo(eeroi))

    return eeroi, eepoint

def szpixelcount(eeimage, eegeometry):
    pixelcount = eeimage.select(0).unmask(sameFootprint=False).reduceRegion(ee.Reducer.count(), eegeometry)
    sz = f"pixelcount: {pixelcount.getInfo()}"
    return sz

__Proba V 333m with reference S2 10m__

In [ ]:
eepoint        = geeutils.tapspoint
eedate         = geeutils.fleecycloudsday

#
# select reference image: s2 ndvi (10m)
#
eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
#
# create reference roi
#
refroidiameter = 64
eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)
#
# target product
#
eesrcimage     = geeutils.somePV333ndviImageNear(eedate, eepoint)
#
# specify target resolution
#
dstroidiameter = int(refroidiameter * 10 / 330) + 2
#
# translate and scale reference crs, and apply it to target
#
eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)

eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))
#
#
#
xin = eesrcimage.clip(eepoint.buffer(5000)).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = eesrcimage.clip(eepoint.buffer(5000)).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print( "pixelcount eerefimage: ", szpixelcount(eerefimage, eeroi) )
print( "pixelcount eesrcimage: ", szpixelcount(eesrcimage, eeroi) )
print( "pixelcount eedstimage: ", szpixelcount(eedstimage, eeroi) )


map = geemap.Map(height='400px')
map.centerObject(eepoint, 15)
map.addLayer(eerefimage, {'min':0,   'max':1},   'eerefimage')
map.addLayer(eesrcimage, {'min':xin, 'max':xax}, 'eesrcimage')
map.addLayer(eedstimage, {'min':xin, 'max':xax}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),             {'palette':'#ff0000'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          20, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 20, 2), {'palette':'#ff0000'}, 'roicenterpoint')
map


__Sentinel 1 VV/VH bands with reference S2 10m__

In [ ]:
eepoint        = geeutils.tapspoint
eedate         = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 5

eesrcimage     = geeutils.someS1rviImageNear(eedate, eepoint)
dstroidiameter = 5

eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)
eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))


#
#
#
xin = eesrcimage.clip(eepoint.buffer(500)).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = eesrcimage.clip(eepoint.buffer(500)).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print( "pixelcount eerefimage: ", szpixelcount(eerefimage, eeroi) )
print( "pixelcount eesrcimage: ", szpixelcount(eesrcimage, eeroi) )
print( "pixelcount eedstimage: ", szpixelcount(eedstimage, eeroi) )


map = geemap.Map(height='400px')
map.centerObject(eepoint, 18)
map.addLayer(eerefimage, {'min':0,   'max':1},   'eerefimage')
map.addLayer(eesrcimage, {'min':xin, 'max':xax}, 'eesrcimage')
map.addLayer(eedstimage, {'min':xin, 'max':xax}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#ff0000'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#ff0000'}, 'roicenterpoint')
map



__Sentinel 1 'angle' band with reference S2 10m__
- compare results for COPERNICUS/S1_GRD and (undocumented?) COPERNICUS/S1_GRD_**FLOAT** collection


In [ ]:
eepoint      = geeutils.tapspoint
eedate       = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 64

if False:
    collection = geeutils.s1rbgImageCollection
else:
    s1floatImageCollection    = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
    s1floatrbgImageCollection = (s1floatImageCollection
                               .filter(ee.Filter.listContains('system:band_names','VV'))
                               .filter(ee.Filter.listContains('system:band_names','VH'))
                               .filter(ee.Filter.listContains('system:band_names','angle'))
                               .select(['VV', 'VH', 'angle']))                                  # False Color - not everywhere available
    collection = s1floatrbgImageCollection
    
eesrcimage     = geeutils.someImageNear(collection.select('angle'), eedate, eepoint)
dstroidiameter = 2 # normally 1 should be enough, but however large the pixels are, they could always intersect in the roi


eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)
eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))
#
#
#
xin = eesrcimage.clip(eepoint.buffer(100000)).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = eesrcimage.clip(eepoint.buffer(100000)).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

print( "pixelcount eerefimage: ", szpixelcount(eerefimage, eeroi) )
print( "pixelcount eesrcimage: ", szpixelcount(eesrcimage, eeroi) )
print( "pixelcount eedstimage: ", szpixelcount(eedstimage, eeroi) )


map = geemap.Map(height='400px')
map.centerObject(eepoint, 12)
map.addLayer(eerefimage, {'min':0,   'max':1},   'eerefimage')
map.addLayer(eesrcimage, {'min':xin, 'max':xax}, 'eesrcimage')
map.addLayer(eedstimage, {'min':xin, 'max':xax}, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#ff0000'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#ff0000'}, 'roicenterpoint')
map

__Sentinel 2 SCL bands with reference S2 10m__
- remark: since s2 10m & 20m grids align, s2 20m should be preferred as reference

In [ ]:
eepoint        = geeutils.bobspoint
eedate         = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 16

eesrcimage     = geeutils.someImageNear(geeutils.s2sclImageCollection, eedate, eepoint)
dstroidiameter = 8


eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)
eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))
#
#
#
print( "pixelcount eerefimage: ", szpixelcount(eerefimage, eeroi) )
print( "pixelcount eesrcimage: ", szpixelcount(eesrcimage, eeroi) )
print( "pixelcount eedstimage: ", szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='400px')
map.centerObject(eepoint, 16)
map.addLayer(eerefimage, {'min':0,  'max':1 },    'eerefimage')
map.addLayer(eesrcimage, geeutils.s2sclvisParams, 'eesrcimage')
map.addLayer(eedstimage, geeutils.s2sclvisParams, 'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#ff0000'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#0000ff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#ff0000'}, 'roicenterpoint')
map


__Sentinel 2 SCL ConvMask band with reference S2 10m__


In [ ]:
eepoint        = geeutils.tapspoint
eedate         = geeutils.fleecycloudsday

eerefimage     = geeutils.someS2ndviImageNear(eedate, eepoint)
refroidiameter = 32 # 34

def convmask(image):
    return ee.Image((geemask.ConvMask( [[2, 4, 5, 6, 7], [3, 8, 9, 10, 11]], [20*9, 20*101] ,[-0.057, 0.025] )
            .makemask(image)
            .unmask(255, False)  # sameFootprint=False: otherwise missing beyond footprint becomes 0
            .toUint8()           # uint8 [0:not masked, 1:masked], no data: 255)
            .rename('MASK')
            .copyProperties(image, ['system:id', 'system:time_start'])))

eeorgimage     = geeutils.someImageNear(geeutils.s2sclImageCollection, eedate, eepoint)
eesrcimage     = convmask(eeorgimage)
dstroidiameter = 16 # 17


eeroi, eeroicenterpoint = squarepixelboundsroi(eepoint, refroidiameter, eerefimage, verbose=False)

eeulx = eeroi.coordinates().flatten().get(0)
eeuly = eeroi.coordinates().flatten().get(1)
eedstimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eeulx, eeuly)
                                  .scale(refroidiameter/dstroidiameter, refroidiameter/dstroidiameter))
#
# image.updateMask(othermask):
# - zet de mask waardes van de image
# - op de othermask waarde
# - maar enkel waar de originele mask niet "0" (masked) is
# - maw: 
#   - er komen enkel mask waarden bij
#   - nieuwe mask = oude mask AND othermask (als we geen fractional mask beschouwen)
#
# geemask.ConvMask maakt image met "1 = cloud"
# convmask.eq(1) geeft 0 (false) waar GEEN wolk was
# image = convmask.updateMask(convmask.eq(1))
# is dan een image die masked is waar GEEN wolken zijn
# en dus worden GEEN wolken transparant
#
# jiezes...
#
eesrcimage = eesrcimage.updateMask(eesrcimage.eq(1))
eedstimage = eedstimage.updateMask(eedstimage.eq(1))

#
#
#
print( "pixelcount eerefimage: ", szpixelcount(eerefimage, eeroi) )
print( "pixelcount eeorgimage: ", szpixelcount(eeorgimage, eeroi) )
print( "pixelcount eesrcimage: ", szpixelcount(eesrcimage, eeroi) )
print( "pixelcount eedstimage: ", szpixelcount(eedstimage, eeroi) )
    
map = geemap.Map(height='400px')
map.centerObject(eepoint, 13) 
map.addLayer(eeorgimage, geeutils.s2sclvisParams,                                          'eeorgimage')
map.addLayer(eerefimage, {'min':0,  'max':1, 'palette':geeutils.ndvivisParamsPalette},     'eerefimage')
map.addLayer(eesrcimage, {'min':0,  'max':1 },                                             'eesrcimage')
map.addLayer(eedstimage, {'min':0,  'max':1 },                                             'eedstimage')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 2),            {'palette':'#ff0000'}, 'roi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#0000ff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#ff0000'}, 'roicenterpoint')
map

